# Assignment Part 1 

### Install BeautifulSoup library to extract information from HTML

In [170]:
!conda install -c conda-forge beautifulsoup4

Solving environment: done

# All requested packages already installed.



### Import necessary libraries

In [171]:
import urllib
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Start reading HTML

In [172]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)

### Cleaning the HTML for data extraction

In [173]:
for script in soup(["script", "style"]):
    script.extract()
td_tag = soup.find_all('td', limit=864)
for tag in td_tag:
    if tag.a:
        tag.a.unwrap()

### Further cleaning and getting all the information

In [174]:
result_list = []
for tag in soup.find_all('td', limit=864):
    result_list.append(tag.text.replace("\n", "").replace("<td>", "").replace("</td>", ""))

data_list=[result_list[i:i+3] for i in range(0, len(result_list), 3)]

### Construct Pandas DataFrame

In [175]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
Toronto = pd.DataFrame(data_list, columns=column_names)

### Cleaning the DataFrame

Ignore cells with a borough that is Not assigned.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [176]:
for i in range(len(Toronto.index)):
    if Toronto.loc[i,"Borough"] == "Not assigned":
        Toronto = Toronto.drop(i)
    elif Toronto.loc[i,"Neighborhood"] == "Not assigned":
        Toronto.loc[i,"Neighborhood"] = Toronto.loc[i,"Borough"]
Toronto.reset_index(drop=True, inplace=True)

### Preparing to get neighborhoods with the same postal code together

In [177]:
postalcode = Toronto["PostalCode"]
postal_list = postalcode.unique().tolist()
count_list=[0 for i in range(len(postal_list))]

### Get the index of the first occurrence of a postal code

In [178]:
for i in range(len(postalcode.index)):
    for j in range(len(postal_list)):
        if Toronto.loc[i, "PostalCode"] == postal_list[j]:
            count_list[j] = i

### Append neighborhoods together for the same postal code, and record rows to be removed

In [179]:
remove_list = []
for i in range(len(postal_list)-1):
    for j in range(count_list[i]+1, count_list[i+1]):
        Toronto.loc[count_list[i], "Neighborhood"] = Toronto.loc[count_list[i], "Neighborhood"] + ", " + Toronto.loc[count_list[i+1],"Neighborhood"]
        remove_list.append(j)

### Remove rows with duplicate postal code

In [180]:
Toronto = Toronto.drop(remove_list)

### Reset index number

In [193]:
Toronto = Toronto.sort_values(by=["PostalCode"])
Toronto.reset_index(drop=True, inplace=True)

### Sort by PostalCode and display the completed DataFrame

In [194]:
Toronto

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern
1,M1C,Scarborough,"Port Union, Don Mills South"
2,M1E,Scarborough,West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,"Scarborough Village, Oriole, Oriole"
6,M1K,Scarborough,Kennedy Park
7,M1L,Scarborough,"Oakridge, York Mills"
8,M1M,Scarborough,"Scarborough Village West, Willowdale"
9,M1N,Scarborough,Cliffside West


### Display the shape of the DataFrame

In [183]:
Toronto.shape

(103, 3)

# Assignment Part 2

### Download the .csv file

In [184]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


### Read the .csv file and load into a Pandas DataFrame

In [197]:
latlon = pd.read_csv('Geospatial_Coordinates.csv')
latlon = latlon.rename(columns={"Postal Code":"PostalCode"})

### Merge the two DataFrame and display the result

In [198]:
Toronto_latlon = pd.merge(Toronto, latlon, on="PostalCode", how="left")
Toronto_latlon

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Don Mills South",43.784535,-79.160497
2,M1E,Scarborough,West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,"Scarborough Village, Oriole, Oriole",43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park,43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, York Mills",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Willowdale",43.716316,-79.239476
9,M1N,Scarborough,Cliffside West,43.692657,-79.264848


# Assignment Part 3 

### Install and import necessary libraries

In [200]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  en

### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [201]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [211]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_latlon['Latitude'], Toronto_latlon['Longitude'], Toronto_latlon['Borough'], Toronto_latlon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Slice the original dataframe and create a new dataframe of areas whose name contains "Toronto".

In [387]:
toronto_data = Toronto_latlon[Toronto_latlon['Borough'].str.contains("Toronto") == True].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, Toronto Dominion Centre",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, Victoria Hotel",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Lawrence Manor East",43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Get the geographical coordinates of Downtown Toronto.

In [388]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


### Visualize Toronto with the neighborhoods in it

In [398]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Here is a hidden cell with Foursquare login credentials.

In [419]:
# The code was removed by Watson Studio for sharing.

### Get the first neighborhood's latitude and longitude values.

In [390]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Get the top 100 venues that are in this neighborhood within a radius of 500 meters.

In [277]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=25WO2PBH2S21W1D4FZUIH2IUTLTJW2I0QXS0WIA1SPRAQGL2&client_secret=VJUADPTP3WXCS5P0HMZ4BULDXXSGDYFIOBXAY2A12PG4WQ1I&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

### Send the GET request and examine the resutls

In [278]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7cf789a30619002cf4d93f'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [279]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json and structure it into a pandas dataframe.

In [280]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


### Check how many venues were returned by Foursquare?

In [281]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Create a function to repeat the same process to all the neighborhoods whose name contains "Toronto".

In [282]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [391]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
Riverdale, Toronto Dominion Centre
India Bazaar, Victoria Hotel
Studio District, Lawrence Manor East
Lawrence Park
Davisville North, Forest Hill West
North Toronto West, Yorkville, Yorkville
Davisville, University of Toronto
Summerhill East, Kensington Market, Kensington Market
Summerhill West, South Niagara, South Niagara, South Niagara, South Niagara, South Niagara, South Niagara
Rosedale
St. James Town, Underground city
Church and Wellesley
Regent Park, Lawrence Manor
Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Dufferin
Union Station, Trinity
Toronto Dominion Centre, Parkdale Village, Parkdale Village
Victoria Hotel, Upwood Park, Upwood Park
Roselawn, Runnymede
Forest Hill West, The Junction South
Yorkville, Roncesvalles
University of Toronto, Swansea
Kensington Market, Steeles East, Steeles East, Steeles East
South Niagara, New Toronto, New Toronto
Stn A PO Boxes 25 The Esplanade, Long Branch
Underground city, Old Mill North, Old Mill North
C

### Check the size of the resulting dataframe

In [392]:
print(toronto_venues.shape)
toronto_venues.head()

(1707, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, Toronto Dominion Centre",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Check how many venues were returned for each neighborhood

In [393]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Business Reply Mail Processing Centre 969 Eastern, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea",17,17,17,17,17,17
Central Bay Street,87,87,87,87,87,87
Christie,16,16,16,16,16,16
Church and Wellesley,87,87,87,87,87,87
"Davisville North, Forest Hill West",7,7,7,7,7,7
"Davisville, University of Toronto",39,39,39,39,39,39
Dufferin,14,14,14,14,14,14
"Forest Hill West, The Junction South",4,4,4,4,4,4


### Find out how many unique categories can be curated from all the returned venues

In [394]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


### Analyze each neighborhood

In [395]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Examine the new dataframe size.

In [396]:
toronto_onehot.shape

(1707, 235)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [397]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000
1,Business Reply Mail Processing Centre 969 East...,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.011494,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.011494,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.011494,0.000000,0.000000,0.011494,0.000000
3,Christie,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.011494,0.011494,0.0000,0.0000,0.000,0.0000,0.000,0.011494,0.000000,...,0.011494,0.000000,0.000000,0.00,0.000000,0.000000,0.011494,0.011494,0.000000,0.011494
5,"Davisville North, Forest Hill West",0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Davisville, University of Toronto",0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.025641,0.000000,...,0.000000,0.025641,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
7,Dufferin,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Forest Hill West, The Junction South",0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Garden District,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.010000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000


### Confirm the new size

In [399]:
toronto_grouped.shape

(38, 235)

### Print each neighborhood along with the top 5 most common venues

In [400]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea, Sunnylea----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Auto Workshop  0.06
3    Recording Studio  0.06
4          Restaurant  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.06
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


----Christie----
               venue  freq
0               Café  0.19
1      Grocery Store  0.19
2               Park  0.12
3  Convenience Store  0.06
4        Coffee Shop  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1              Gay Bar  0.06
2  Japanese Restaurant  0.06
3     Sushi Restaurant  0

### Put into a pandas dataframe

In [401]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [402]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dttoronto_grouped['Neighborhood']

for ind in np.arange(dttoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Beer Bar,Bakery,Seafood Restaurant,Italian Restaurant
1,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Farmers Market,Restaurant,Burrito Place,Auto Workshop,Spa,Fast Food Restaurant,Garden Center,Garden,Pizza Place
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Salad Place,Bubble Tea Shop,Spa
3,Christie,Grocery Store,Café,Park,Diner,Convenience Store,Restaurant,Baby Store,Coffee Shop,Italian Restaurant,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


### Cluster Neighborhood

Run k-means to cluster the neighborhood into 9 clusters. **(Why 9 clusters?)**

In [403]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18]

array([2, 2, 2, 0, 2, 2, 2, 0, 7, 2, 2, 2, 4, 2, 2, 2, 2, 2], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [404]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
1,M4K,East Toronto,"Riverdale, Toronto Dominion Centre",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"India Bazaar, Victoria Hotel",43.668999,-79.315572,2,Park,Sandwich Place,Pet Store,Pub,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,East Toronto,"Studio District, Lawrence Manor East",43.659526,-79.340923,2,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Cheese Shop,Stationery Store,Bar,Middle Eastern Restaurant,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Display the map with cluster labels

In [405]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check Cluster 1

In [406]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,0,Grocery Store,Café,Park,Diner,Convenience Store,Restaurant,Baby Store,Coffee Shop,Italian Restaurant,Nightclub
31,West Toronto,0,Pharmacy,Supermarket,Bakery,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery,Bar,Bank
34,West Toronto,0,Mexican Restaurant,Café,Grocery Store,Furniture / Home Store,Music Venue,Diner,Discount Store,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore


### Check Cluster 2

In [408]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Wings Joint,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Check Cluster 3 

In [409]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,East Toronto,2,Park,Sandwich Place,Pet Store,Pub,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,East Toronto,2,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Cheese Shop,Stationery Store,Bar,Middle Eastern Restaurant,Chinese Restaurant
5,Central Toronto,2,Gym,Park,Sandwich Place,Breakfast Spot,Clothing Store,Food & Drink Shop,Hotel,Donut Shop,Dog Run,Doner Restaurant
6,Central Toronto,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Park,Salon / Barbershop,Burger Joint,Restaurant,Rental Car Location,Café,Chinese Restaurant
7,Central Toronto,2,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Gym,Coffee Shop,Thai Restaurant,Pizza Place,Sushi Restaurant,Seafood Restaurant
9,Central Toronto,2,Coffee Shop,Pub,Bagel Shop,Restaurant,Vietnamese Restaurant,Liquor Store,Supermarket,Pizza Place,Sushi Restaurant,American Restaurant
11,Downtown Toronto,2,Coffee Shop,Restaurant,Chinese Restaurant,Pizza Place,Café,Market,Pet Store,Italian Restaurant,Bakery,Pub
12,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
13,Downtown Toronto,2,Coffee Shop,Café,Pub,Bakery,Park,Gym / Fitness Center,Mexican Restaurant,Theater,Restaurant,Breakfast Spot


### Check Cluster 4

In [410]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Health & Beauty Service,Music Venue,Garden,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Check Cluster 5

In [411]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Bus Line,Swim School,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Check Cluster 6

In [412]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,5,Park,Playground,Trail,Building,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Check Cluster 7

In [413]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,6,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport,Airport Food Court,Boutique,Boat or Ferry,Bar


### Check Cluster 8

In [414]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,7,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Check Cluster 9

In [415]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,8,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant


#### **The reason why 9 clusters were used is because 9 is the smallest size to separate Cluster 7 - Airport from the others.**

### So we decided the names to describe these clusters, and add to the DataFrame

In [416]:
Names = pd.Series(["Shopping", "Playground", "Eatery", "Health & Beauty", "Lawrence Park", "Rosedale Park", "Airport", "Rich", "Beach"])
for i in range(len(toronto_merged.index)):
    toronto_merged.loc[i, 'Cluster Names'] = Names[toronto_merged.loc[i, 'Cluster Labels']]

### Check the dataframe for one more time

In [417]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Names
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant,Beach
1,M4K,East Toronto,"Riverdale, Toronto Dominion Centre",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant,Eatery
2,M4L,East Toronto,"India Bazaar, Victoria Hotel",43.668999,-79.315572,2,Park,Sandwich Place,Pet Store,Pub,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Eatery
3,M4M,East Toronto,"Studio District, Lawrence Manor East",43.659526,-79.340923,2,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Cheese Shop,Stationery Store,Bar,Middle Eastern Restaurant,Chinese Restaurant,Eatery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Lawrence Park
5,M4P,Central Toronto,"Davisville North, Forest Hill West",43.712751,-79.390197,2,Gym,Park,Sandwich Place,Breakfast Spot,Clothing Store,Food & Drink Shop,Hotel,Donut Shop,Dog Run,Doner Restaurant,Eatery
6,M4R,Central Toronto,"North Toronto West, Yorkville, Yorkville",43.715383,-79.405678,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Park,Salon / Barbershop,Burger Joint,Restaurant,Rental Car Location,Café,Chinese Restaurant,Eatery
7,M4S,Central Toronto,"Davisville, University of Toronto",43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Gym,Coffee Shop,Thai Restaurant,Pizza Place,Sushi Restaurant,Seafood Restaurant,Eatery
8,M4T,Central Toronto,"Summerhill East, Kensington Market, Kensington...",43.689574,-79.383160,1,Playground,Wings Joint,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Playground
9,M4V,Central Toronto,"Summerhill West, South Niagara, South Niagara,...",43.686412,-79.400049,2,Coffee Shop,Pub,Bagel Shop,Restaurant,Vietnamese Restaurant,Liquor Store,Supermarket,Pizza Place,Sushi Restaurant,American Restaurant,Eatery


# Final Result

In [418]:
# create map
map_clusters_names = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, name in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'], toronto_merged['Cluster Names']):
    label = folium.Popup(str(poi) + ' - ' + str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_names)
       
map_clusters_names